<a href="https://colab.research.google.com/github/shibukawar/stein-importance-sampling-qa/blob/feature%2Fstein-importance-sampling/notebook/example_stein_importance_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments on free energy and magnetization

In [ ]:
import os

os.chdir("../")

In [ ]:
import numpy as np

import sqlite3

from stein.energy import create_hamiltonian
from stein.util import read_from_file

In [ ]:
PROBLEM = "GSD_6"

In [ ]:
PROBLEM_DIR_PATH = f"./problems/{PROBLEM}/"

J, h = read_from_file(PROBLEM_DIR_PATH)
hamiltonian = create_hamiltonian(J, h)

# Define a target distribution

In [ ]:
init_table = False
if not os.path.exists('experiments.db'):
    init_table = True
con = sqlite3.connect("experiments.db")

if init_table is True:
    cur = con.cursor()
    cur.execute("CREATE TABLE exact_free_energy(problem, beta, energy)")
    cur.execute("CREATE TABLE exact_magnetization(problem, beta, magnetization)")
    cur.execute("CREATE TABLE exact_susceptibility(problem, beta, susceptibility)")
    cur.execute("CREATE TABLE exact_m2_m4(problem, beta, m2, m4)")
    con.commit()
    con.close()

In [ ]:
def m2_m4(beta, hamiltonian):
    dim = 16
    exact_m2, exact_m4 = 0, 0
    Z = 0
    for n in range(2**dim):
        b = np.binary_repr(n, width=dim)
        sigma = np.array(list(map(lambda x: 2 * int(x) - 1, list(b))))
        energy = hamiltonian(sigma)
        exact_m4 *= Z
        exact_m2 *= Z
        Z += np.exp(-beta * energy)
        exact_m4 += sum(sigma) ** 4 * np.exp(-beta * energy)
        exact_m2 += sum(sigma) ** 2 * np.exp(-beta * energy)
        exact_m4 /= Z
        exact_m2 /= Z
    return exact_m2, exact_m4


print(PROBLEM)
# for beta in np.arange(0.1, 1.1, 0.1):
for beta in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    beta = float(beta)
    print(beta)
    m2, m4 = m2_m4(beta, hamiltonian)
    con = sqlite3.connect("experiments.db")
    cur = con.cursor()
    cur.execute(
        f"INSERT INTO exact_m2_m4(problem, beta, m2, m4) VALUES(?, ?, ?, ?)",
        (PROBLEM, beta, m2, m4),
    )
    con.commit()
    con.close()

In [ ]:
def ene_and_mag(beta, hamiltonian):
    dim = 16
    exact_free_energy = 0
    exact_mag = 0
    exact_susceptibility = 0
    Z = 0
    for n in range(2**dim):
        b = np.binary_repr(n, width=dim)
        sigma = np.array(list(map(lambda x: 2 * int(x) - 1, list(b))))
        energy = hamiltonian(sigma)
        exact_free_energy *= Z
        exact_mag *= Z
        exact_susceptibility *= Z
        Z += np.exp(-beta * energy)
        exact_free_energy += energy * np.exp(-beta * energy)
        exact_free_energy /= Z
        exact_mag += sum(sigma) * np.exp(-beta * energy) / dim
        exact_susceptibility += sum(sigma) ** 2 * beta * np.exp(-beta * energy)
        exact_mag /= Z
        exact_susceptibility /= Z
    return exact_free_energy, exact_mag, exact_susceptibility


# for beta in np.arange(0.1, 1.1, 0.1):
for beta in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    beta = float(beta)
    print(beta)
    ene, mag, s = ene_and_mag(beta, hamiltonian)
    con = sqlite3.connect("experiments.db")
    cur = con.cursor()
    cur.execute(
        f"INSERT INTO exact_free_energy(problem, beta, energy) VALUES(?, ?, ?)",
        (PROBLEM, beta, ene),
    )
    cur.execute(
        f"INSERT INTO exact_magnetization(problem, beta, magnetization) VALUES(?, ?, ?)",
        (PROBLEM, beta, mag),
    )
    cur.execute(
        f"INSERT INTO exact_susceptibility(problem, beta, susceptibility) VALUES(?, ?, ?)",
        (PROBLEM, beta, s),
    )
    con.commit()
    con.close()